In [39]:
from SDP_interaction_inference.constraints import Constraint
from SDP_interaction_inference import optimization
from SDP_interaction_inference.dataset import Dataset
from SDP_interaction_inference.correlation import Correlation
from SDP_interaction_inference import utils
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import json
import scipy
import tqdm
from copy import deepcopy
import matplotlib.patches
import corner
import plotly
import plotly.graph_objects as go

# Model Free Investigation

## Data Loading

In [41]:
# read fibroblast transcript counts
data_FIB = pd.read_csv("./Data/GSE151334_FIB_counts_thresh.csv", index_col=0)

# load capture
beta = np.loadtxt("./Capture/beta_FIB.txt")

# load RNA types
biotypes_dict = json.load(open("./Biotypes/biotypes_FIB.json"))

# select indices of protein coding mRNA and non-coding miRNA
pcRNA_indices = [idx for idx, btype in enumerate(biotypes_dict.values()) if btype == "protein_coding"]
miRNA_indices = [idx for idx, btype in enumerate(biotypes_dict.values()) if btype == "miRNA"]

# separate data
data_FIB_pcRNA = data_FIB.iloc[pcRNA_indices]
data_FIB_miRNA = data_FIB.iloc[miRNA_indices]

# amounts: see './Biotypes/updated-biotypes.ipynb' for full pie chart of types
print(f"Total of {data_FIB.shape[0]} genes with mean > 1 present \nSelected {data_FIB_miRNA.shape[0]} non-coding miRNA \nSelected {data_FIB_pcRNA.shape[0]} protein coding (mRNA)")

Total of 11747 genes with mean > 1 present 
Selected 19 non-coding miRNA 
Selected 9296 protein coding (mRNA)


## Setup functions

In [10]:
def construct_dataset(mirna_sample, mrna_dataset, beta, resamples=1000):

    # size
    gene_pairs, cells = mrna_dataset.shape

    # construct paired count dataframe
    counts_df = pd.DataFrame(
        index = [f"Gene-pair-{i}" for i in range(gene_pairs)],
        columns = [f"Cell-{j}" for j in range(cells)]
    )

    # fill with pairs
    for i in range(gene_pairs):
        gene_i = mirna_sample
        gene_j = mrna_dataset.iloc[i]
        gene_pair_ij = list(zip(gene_i, gene_j))
        counts_df.iloc[i] = gene_pair_ij

    # construct dataset object
    data = Dataset()
    data.count_dataset = counts_df
    data.cells = cells
    data.gene_pairs = gene_pairs

    # settings
    data.resamples = resamples

    # set capture
    data.beta = beta

    return data

In [11]:
def model_free_independence_test(dataset, d, **kwargs):

    # Independent model free
    constraints = Constraint(
        moment_bounds=True,
        moment_matrices=True,
        factorization=True
    )
    opt_MF_ind = optimization.ModelFreeOptimization(dataset, d, constraints=constraints, printing=False, silent=True, **kwargs)
    opt_MF_ind.analyse_dataset()

    # extract results
    status = [solution['status'] for solution in opt_MF_ind.result_dict.values()]
    times  = [solution['time'] for solution in opt_MF_ind.result_dict.values()]
    cuts   = [solution['cuts'] for solution in opt_MF_ind.result_dict.values()]

    # collect
    data = {
        'status': status,
        'times': times,
        'cuts': cuts,
        'model': opt_MF_ind
    }

    return data

In [12]:
def model_free_interacting_test(dataset, d, **kwargs):

    # Interacting model free
    constraints = Constraint(
        moment_bounds=True,
        moment_matrices=True,
        factorization=False
    )
    opt_MF_int = optimization.ModelFreeOptimization(dataset, d, constraints=constraints, printing=False, silent=True, **kwargs)
    opt_MF_int.analyse_dataset()

    # extract results
    status = [solution['status'] for solution in opt_MF_int.result_dict.values()]
    times  = [solution['time'] for solution in opt_MF_int.result_dict.values()]
    cuts   = [solution['cuts'] for solution in opt_MF_int.result_dict.values()]
    correlation = [solution['correlation'] for solution in opt_MF_int.result_dict.values()]

    # collect
    data = {
        'status': status,
        'times': times,
        'cuts': cuts,
        'correlation': correlation,
        'model': opt_MF_int
    }

    return data

## Setup results

### Construct

In [13]:
# select mRNA
mRNA = data_FIB_pcRNA.index

In [ ]:
# dataframe to store model free results
#MF_independent_df = pd.DataFrame(
#    index=mRNA
#)
#MF_interacting_df = pd.DataFrame(
#    index=mRNA
#)

### Load

In [ ]:
#MF_independent_df = pd.read_csv("Results/independent_MF.csv", index_col=0)
#MF_interacting_df = pd.read_csv("Results/interacting_MF.csv", index_col=0)

### Save

In [ ]:
#MF_independent_df.to_csv("Results/independent_MF.csv")
#MF_interacting_df.to_csv("Results/interacting_MF.csv")

## Running

In [ ]:
# for each miRNA
miRNA_names = data_FIB_miRNA.index
for miRNA in miRNA_names:

    # display miRNA
    print(f"\nRunning {miRNA}")

    # construct dataset of miRNA paired with mRNA
    dataset_SDP = construct_dataset(data_FIB_miRNA.loc[miRNA], data_FIB_pcRNA, beta_read)

    # bootstrap
    print("Bootstrap:")
    dataset_SDP.confidence = 0.95
    dataset_SDP.bootstrap(d=4, tqdm_disable=False)

    # independence test
    print("MF Independence test:")
    MF_ind_d4_c95 = model_free_independence_test(dataset_SDP, d=4, time_limit=30, total_time_limit=30)

    # display results
    print(np.unique(MF_ind_d4_c95['status'], return_counts=True))

    # interacting test
    print("MF Interacting test:")
    MF_int_d4_c95 = model_free_interacting_test(dataset_SDP, d=4, time_limit=30, total_time_limit=30)

    # display results
    print(np.unique(MF_int_d4_c95['status'], return_counts=True))

    # store results on dataframe

    # independence test
    MF_independent_df[f'{miRNA}_d4_c95_status'] = MF_ind_d4_c95['status']
    MF_independent_df[f'{miRNA}_d4_c95_times'] = MF_ind_d4_c95['times']
    MF_independent_df[f'{miRNA}_d4_c95_cuts'] = MF_ind_d4_c95['cuts']

    # interacting test
    MF_interacting_df[f'{miRNA}_d4_c95_status'] = MF_int_d4_c95['status']
    MF_interacting_df[f'{miRNA}_d4_c95_times'] = MF_int_d4_c95['times']
    MF_interacting_df[f'{miRNA}_d4_c95_cuts'] = MF_int_d4_c95['cuts']
    MF_interacting_df[f'{miRNA}_d4_c95_correlation'] = MF_int_d4_c95['correlation']

    # save dataframes as safety
    #MF_independent_df.to_csv("Results/independent_MF.csv")  # Uncomment when running
    #MF_interacting_df.to_csv("Results/interacting_MF.csv")


Running MIR100
Bootstrap:


100%|██████████| 9376/9376 [05:57<00:00, 26.25it/s]


MF Independence test:


100%|██████████| 9376/9376 [08:40<00:00, 18.02it/s]


(array(['CUT_LIMIT', 'INFEASIBLE', 'OPTIMAL'], dtype='<U10'), array([  10, 1926, 7440]))
MF Interacting test:


...

MF Independence test:


100%|██████████| 9376/9376 [10:15<00:00, 15.23it/s]


(array(['CUT_LIMIT', 'INFEASIBLE', 'OPTIMAL'], dtype='<U10'), array([  13, 1696, 7667]))
MF Interacting test:


 71%|███████   | 6671/9376 [08:16<04:10, 10.78it/s]c:\Users\willi\Documents\ProjectPaper\M5R_venv\lib\site-packages\SDP_interaction_inference\optimization_utils.py:615: RuntimeWarning: invalid value encountered in sqrt
  correlation = cov_xy / (np.sqrt(var_x) * np.sqrt(var_y))
100%|██████████| 9376/9376 [11:38<00:00, 13.42it/s]


(array(['CUT_LIMIT', 'INFEASIBLE', 'OPTIMAL'], dtype='<U10'), array([1579,   87, 7710]))


## Notes

- some NUMERIC status
- varying levels of dependence
- overall a lot of CUT_LIMIT for interacting test, and some INFEASIBLE